
A FASTQ file normally uses four lines per sequence.

- Line 1 begins with a '@' character and is followed by a series of information that uniquely identifies each read.

@HWI-ST1097:127:C0W5VACXX:5:1101:4820:2124 1:N:0:CTCAGA
@SRR534005.299 HWI-ST945:93:c02g4acxx:3:1101:16012:2225/1

- Line 2 is the raw sequence. The lenght of this sequence should correspond to the length of the read requested.

- Line 3 is always '+' for output from the Illumina.

- Line 4 is a string of Ascii-encoded base quality scores.

# Running trimmomatic in a loop
for R1 in *R1*
do
   R2=${R1//R1_001_small.fastq/R2_001_small.fastq}
   R1paired=${R1//.fastq/_paired.fastq.gz}
   R1unpaired=${R1//.fastq/_unpaired.fastq.gz}	
   R2paired=${R2//.fastq/_paired.fastq.gz}
   R2unpaired=${R2//.fastq/_unpaired.fastq.gz}	
   echo "java -jar $WORK/IntMolModule/STG/practiceData/Trimmomatic-0.36/trimmomatic-0.36.jar PE -threads 32 -phred33 $R1 $R2 $R1paired $R1unpaired $R2paired $R2unpaired" >> trimmomatic.cmds
done
cat trimmomatic.cmds

wget http://kandurilab.org/users/santhilal/courses/UNIX/materials/course_file_repo.zip

Onedrive
wget -qO - https://download.opensuse.org/repositories/home:/npreining:/debian-ubuntu-onedrive/xUbuntu_22.04/Release.key | gpg --dearmor | sudo tee /usr/share/keyrings/obs-onedrive.gpg > /dev/null

echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/obs-onedrive.gpg] https://download.opensuse.org/repositories/home:/npreining:/debian-ubuntu-onedrive/xUbuntu_22.04/./" | sudo tee /etc/apt/sources.list.d/onedrive.list


Cat  | xargs Rscript --vanilla pipeline.R 

#Read the sample from the command line
sranumber<-commandArgs(trailingOnly=TRUE) 
print(sranumber)
#Function Idk yet
execute<-function(x, outputfile=NA, intern=FALSE, quitOnError=FALSE){
  if(!is.na(outputfile) &&file.exists(outputfile)){
    cat("Output for step exists, skipping this step\n");
    invisible("")
  }
  cat("----", x, "\n"); res<-system(x, intern=intern); cat(">>>>", res[1], "\n")
  if(res[1] >=1){ 
    cat("Error external process did not finish\n\n");
    if(quitOnError) q("no")
  }
}

input.dir<-"/home/dev105/projects/rawdata"
input.base<-sranumber[1]
output.dir<-paste0("/home/dev105/projects/outputdata/", input.base,".aln")

#Create an output folder
if(!file.exists(input.dir)){ dir.create(input.dir, recursive=TRUE) }
if(!file.exists(output.dir)){ dir.create(output.dir, recursive=TRUE) }

#STEP 0 - SRA Download and Compress
setwd(input.dir)
execute(paste0(" -p", input.base)
execute(paste0("fasterq-dump -p --split-files ", input.base), paste0(input.base, "_1.fastq"))
execute(paste0("bgzip ", input.base, "_1.fastq"), paste0(input.base, "_1.fastq.gz"))
execute(paste0("bgzip ", input.base, "_2.fastq"), paste0(input.base, "_2.fastq.gz"))

#STEP 1 - READ Trimming
trim.files<-c(
                  paste0(input.dir, "/", input.base,"_1.fastq.gz"),
                  paste0(input.dir, "/", input.base,"_2.fastq.gz"),
                  paste0(output.dir, "/", input.base,"_1.P.fastq.gz"),
                  paste0(output.dir, "/", input.base,"_1.U.fastq.gz"),
                  paste0(output.dir, "/", input.base,"_2.P.fastq.gz"),
                  paste0(output.dir, "/", input.base,"_2.U.fastq.gz")
                )
trim.path<-"/home/dev105/devapps/Trimmomatic-0.39"
trim.exec<-paste0("java -jar ", trim.path, "/trimmomatic-0.39.jar")
trim.opts<-paste0("ILLUMINACLIP:",trim.path,"/adapters/TruSeq3-PE-2.fa:2:30:10")
trim.opts<-paste0(trim.opts, "LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36")
trim.cmd<-paste0(trim.exec, "PE ", paste0(trim.files, collapse=""), "", trim.opts)

execute(trim.cmd, trim.files[3])

q("no")